In [ ]:
#to install package
!pip install requests
!pip install scrapy
!pip install fuzzywuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.9 MB/s eta 0:00:00


In [ ]:
import scrapy
import requests#import the library
import time
import pandas as pd
from bs4 import BeautifulSoup#import the library
import scrapy.crawler as crawler
from multiprocessing import Process, Queue
from twisted.internet import reactor

In [ ]:

base_url = "https://www.amazon.com"
start_url = "https://www.amazon.com/s?k=computer&page={}"
headers = {
    'cookie': 'session-id=143-0518965-4583815; session-id-time=2082787201l; i18n-prefs=USD; ubid-main=130-9844280-0541435; lc-main=en_US; session-token=AfpLxMSPrFphOhmWJla7aLRlyNF2GqVtXJ2TsEp3okDtJuPb5AeGQlucO9lzKwA/hIrPHj6mQGddILUK704gLg88bRVBz19o+kLS91o1tNLiygylLKsantdEjYYitigTMKJU4TLt8fm2dTc9QG/TiVNZgJLiIWj6Jh2uwkXzTr8QAnV6LHvhKOE9koVCRIkvaZqEn6nPFAbPJ72fGv60WwvJHE50ogKbNvPZWCME7vd8h7e7fLaoIaedVqJgpv73A+71CNGUG7M4bUOuwf6I0r8X5vq04zYApd+4bkrIVwqdHrNxvJEdafdoghpzjVu7bLvJqSyG+JInbXtlXlM7lLgiZWHDw/+P; csm-hit=tb:B46NJC3PAA4MHP8VWW0P+s-B46NJC3PAA4MHP8VWW0P|1738232057645&t:1738232057646&adb:adblk_no',
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36 Edg/132.0.0.0'
}



In [ ]:
# Storing product data
product_data = []
page = 1

while len(product_data) < 100:  # The goal is to crawl 100 pieces of data
    print(f"Crawling page {page}...")

    response = requests.get(start_url.format(page), headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    products = soup.find_all('div', {'data-component-type': 's-search-result'})
    for product in products:
        # Get product title
        title_tag = product.find('h2', class_='a-size-medium')
        title = title_tag.text.strip() if title_tag else "No Title"

        # Get product link
        link_tag = product.find('a', class_='a-link-normal s-no-outline')
        product_link = f"{base_url}{link_tag['href']}" if link_tag else "No Link"

        # Get product image URL
        image_tag = product.find('img', class_='s-image')
        image_url = image_tag['src'] if image_tag else "No Image URL"

        # Get Price
        price_whole = product.find('span', class_='a-price-whole')
        price_fraction = product.find('span', class_='a-price-fraction')
        price = f"{price_whole.text.strip()}.{price_fraction.text.strip()}" if price_whole and price_fraction else "No Price"

        # Get brands, ratings, global ratings, reviews, options
        if product_link != "No Link":
            time.sleep(1)  # Avoid requesting too quickly
            detail_response = requests.get(product_link, headers=headers)
            detail_soup = BeautifulSoup(detail_response.content, "html.parser")

            # Get Brand
            brand_row = detail_soup.find('tr', class_='a-spacing-small po-brand')
            brand_name = brand_row.find('td', class_='a-span9').text.strip() if brand_row else "No Brand"

            # Get ratings
            rating_span = detail_soup.find('span', {'data-hook': 'rating-out-of-text'})
            rating = rating_span.text.strip() if rating_span else "No Rating"

            # Get global ratings
            global_rating_span = detail_soup.find('span', {'data-hook': 'total-review-count'})
            global_rating = global_rating_span.text.strip() if global_rating_span else "No Global Rating"

            # Get comments (up to 3)
            reviews = detail_soup.find_all('span', {'data-hook': 'review-body'})
            review_texts = [review.text.strip() for review in reviews][:3]

            # Get "Options" information
            options_div = detail_soup.find('div', class_='a-section a-spacing-small')
            options = options_div.text.strip() if options_div else "No Options"
        else:
            brand_name = "No Link"
            rating = "No Rating"
            global_rating = "No Global Rating"
            review_texts = []
            options = "No Options"

        # Storing Data
        product_data.append({
            'Title': title,
            'Brand': brand_name,
            'Price': price,
            'Ratings': rating,
            'Global Ratings': global_rating,
            'Reviews': review_texts,
            'Options': options,
            'Product URL': product_link,
            'Image URL': image_url
        })

        if len(product_data) >= 100:
            break  # Stop when 100 data items are reached

    page += 1

df = pd.DataFrame(product_data)
print(df.head(10))
df.to_csv("amazon_products_final.csv", index=False)

Crawling page 1...
Crawling page 2...
Crawling page 3...
Crawling page 4...
Crawling page 5...
Crawling page 6...
Crawling page 7...
                                               Title   Brand    Price  \
0  HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...      HP  170..95   
1  HP 22" All-in-One Desktop PC • The New Version...      HP  549..99   
2  HP Elite Desktop PC Computer Intel Core i5 3.1...      HP  134..00   
3  Dell Optiplex 7050 SFF Desktop PC Intel i7-770...    Dell  228..65   
4  HP Portable Laptop, Student and Business, 14" ...      HP  204..98   
5  ASUS E410 Intel Celeron N4020 4GB 64GB 14-Inch...    ASUS  138..00   
6  HP All-in-One Desktop Computer, 21.5" FHD Disp...      HP  409..99   
7  HP Pro Tower 290 G9 Business Desktop, 12th Gen...      HP  499..99   
8  Lenovo ThinkCentre AIO 24" FHD IPS All-in-One ...  Lenovo  499..99   
9  KAMRUI GK3Plus Mini PC with Windows 11Pro, 16G...  KAMRUI  199..99   

        Ratings        Global Ratings  \
0    4 out of 5  2,174

In [ ]:
df.head(10)


,Title,Brand,Price,Ratings,Global Ratings,Reviews,Options,Product URL,Image URL
0,"HP 14 Laptop, Intel Celeron N4020, 4 GB RAM, 6...",HP,170..95,4 out of 5,"2,174 global ratings",[Needed a computer for genealogy. Works good f...,"Shipping cost, delivery date, and order total ...",https://www.amazon.com/HP-Micro-edge-Microsoft...,https://m.media-amazon.com/images/I/815uX7wkOZ...
1,"HP 22"" All-in-One Desktop PC • The New Version...",HP,549..99,4.2 out of 5,109 global ratings,"[The computer works well\nRead more, I like th...","Shipping cost, delivery date, and order total ...",https://www.amazon.com/HP-Desktop-Microsoft-Ex...,https://m.media-amazon.com/images/I/718sJWwVnb...
2,HP Elite Desktop PC Computer Intel Core i5 3.1...,HP,134..00,3.8 out of 5,"7,581 global ratings","[Not bad for the money. It was a clean, scrat...","Shipping cost, delivery date, and order total ...",https://www.amazon.com/HP-Desktop-Computer-Pac...,https://m.media-amazon.com/images/I/718sn7oOcf...
3,Dell Optiplex 7050 SFF Desktop PC Intel i7-770...,Dell,228..65,3.9 out of 5,"2,324 global ratings",[These del 7050 renewed computers are a great ...,"Shipping cost, delivery date, and order total ...",https://www.amazon.com/Dell-Optiplex-Desktop-E...,https://m.media-amazon.com/images/I/61FVIijQp2...
4,"HP Portable Laptop, Student and Business, 14"" ...",HP,204..98,4.1 out of 5,"2,087 global ratings","[Definitely a slower processor, but is good fo...","Shipping cost, delivery date, and order total ...",https://www.amazon.com/HP-Portable-Student-Bus...,https://m.media-amazon.com/images/I/61IpRGnny7...
5,ASUS E410 Intel Celeron N4020 4GB 64GB 14-Inch...,ASUS,138..00,4 out of 5,748 global ratings,[[REVISED Apr 6 '24 for storage issues - see e...,"Shipping cost, delivery date, and order total ...",https://www.amazon.com/Intel-Celeron-N4020-14-...,https://m.media-amazon.com/images/I/614Jk1dIoG...
6,"HP All-in-One Desktop Computer, 21.5"" FHD Disp...",HP,409..99,4.4 out of 5,35 global ratings,"[If you remember the Jetsons TV show, this rem...","Shipping cost, delivery date, and order total ...",https://www.amazon.com/HP-Desktop-Computer-Pro...,https://m.media-amazon.com/images/I/81qiF6IDKi...
7,"HP Pro Tower 290 G9 Business Desktop, 12th Gen...",HP,499..99,4.2 out of 5,248 global ratings,[The price was great for the features availabl...,"Shipping cost, delivery date, and order total ...",https://www.amazon.com/HP-290-G9-Processor-2-M...,https://m.media-amazon.com/images/I/61Is9E929i...
8,"Lenovo ThinkCentre AIO 24"" FHD IPS All-in-One ...",Lenovo,499..99,4.9 out of 5,44 global ratings,[Bought this computer for my business. Very ea...,"Shipping cost, delivery date, and order total ...",https://www.amazon.com/Lenovo-ThinkCentre-Comp...,https://m.media-amazon.com/images/I/71AiuLNM+P...
9,"KAMRUI GK3Plus Mini PC with Windows 11Pro, 16G...",KAMRUI,199..99,4.5 out of 5,"1,065 global ratings",[I have had my new KAMRUI GK3 PLUS for several...,"Shipping cost, delivery date, and order total ...",https://www.amazon.com/KAMRUI-GK3-2-5-inch-Eth...,https://m.media-amazon.com/images/I/61x9n4E3a7...
